 <h1> <font color='green'>P</font>hylogeny <font color='green'>E</font>mbedding  & </h1> <h1> <font color='green'>A</font>pproximate <font color='green'>R</font>epresentation </h1>
 <img src="https://github.com/AndreaRubbi/Pear-EBI/raw/pear_ebi/logos/LOGO_PEAR.png" width="100" height="100" style='position:absolute; right:500px; top:-15px'>

****
Here we report the code used to generate plots similar to the examples in the manuscript, providing a general outline of the use of the python library

`tree_set` is the main module of `pear_ebi` - it contains both the <font color="blue"><b>tree_set</b></font> and the <font color="red"><b>set_collection</b></font> objects.

In [1]:
from pear_ebi import tree_set
import numpy as np
import pandas as pd
import time
import os

## Examples
Here we do not delve into the specifics or the rationale of the experiments - for that we redirect you to the related manuscript - but rather focus on the use of the library to analyze sets of trees in different settings.

***
# Bayesian Markov Chain Monte Carlo
### <font color='purple'> using trees from BEAST </font> 

This is a relatively just simple example: we have a few files containing trees in Newick format, where each set of trees is produced by a program (BEAST) that produced them sequentially, analyzing a large alignment of SARS-CoV-2 genomes. We want to represent the distribution of these trees in order to analyze the individual trajectories and to compare the different trajectories coming from different runs. <br>
With this idea in mind, we simply compute the square distance matrix using the Robinson-Foulds metric, which generally represents the relations between trees in an effective way, using the most efficient algorithm implemented in Pear: `hashrf`.<br>
We then compute and plot the `PCoA` embeddings in 2 and 3 dimensions. 

#### Step 1: load trees

<font color="blue"><b>tree_set</b></font> is meant to store and manage the analysis of a single set of trees stored in the same file. These trees have to be in the (standard) Newick format. 

`tree_set = tree_set.tree_set(file, output_file=None, distance_matrix=None, metadata=None)`

The first argument indicates the file where the trees are stored. `output_file` can be empty and may be used to indicate a specific name and path for the output distance matrix. `distance_matrix` can be used to indicate a precomputed distance matrix in the `.csv` format. `metadata` may be used to indicate a `.csv` file with additional information relative to the trees. It must be of the same dimensionality of the set of trees, hence $size = (#trees, #meta-variables)$. 
 ***
<font color="red"><b>set_collection</b></font> performs the same tasks of <font color="blue"><b>tree_set</b></font>, but stores multiple set of trees. <br> 
<b>NB:</b> for each file, a set of trees is defined within the set collection. In practice, a <font color="red"><b>set_collection</b></font> is a collection of different <font color="blue"><b>tree_set</b></font>s, each one containing a set of trees coming from a different file. It should not come as a surprise, then, that the input `collection=list()`, potentially empty (you may initialize an empty collection), is a list of files containing trees in the Newick format. 

`set_collection=tree_set.set_collection(collection=list(), file="Set_collection_", output_file=None, distance_matrix=None, metadata=None)`

`file` may be used to indicate an alternative name for the file containing the collection information. This file will be used by Pear to store information relative to the sets of trees. `output_file` may be used to indicate a specific name for the distance matrix file.
<br>`distance_matrix` can be used to indicate a precomputed distance matrix in the `.csv` format. `metadata` may be used to indicate a `.csv` file with additional information relative to the trees. It must be of the same dimensionality of the set of trees, hence $size = (#trees, #meta-variables)$.

In [6]:
beast_dir = '../beast_trees'
# use 1 longer BEAST run and 3 shorter ones (each has been downsampled to 1001 trees) #
beast_runs = [os.path.join(beast_dir, f"beast_long.trees")] + [os.path.join(beast_dir, f"beast_run{i}.trees") for i in range(1,4)]
beast_collection = tree_set.set_collection(beast_runs)
print(beast_collection)

─────────────────────────────            
 Tree set collection containing 4004 trees;            
 File: Set_collection_e285d422-2569-42cd-90d0-a96877dc6308;
 Distance matrix: not computed.                
───────────────────────────── 
beast_long; Containing 1001 trees. 
beast_run1; Containing 1001 trees. 
beast_run2; Containing 1001 trees. 
beast_run3; Containing 1001 trees. 



#### Step 2: compute distances

In [9]:
# distance method can be chosen among hashrf_RF, #
# hashrf_wRF, smart_RF, tqdist_quartet, tqdist_triplet #
beast_collection.calculate_distances(method="hashrf_RF") 
# confirm distances are calculated (check the "Distance matrix:" output) #
print(beast_collection)

Output()

hashrf_RF | Done!

─────────────────────────────            
 Tree set collection containing 4004 trees;            
 File: Set_collection_e285d422-2569-42cd-90d0-a96877dc6308;
 Distance matrix: computed.                
───────────────────────────── 
beast_long; Containing 1001 trees. 
beast_run1; Containing 1001 trees. 
beast_run2; Containing 1001 trees. 
beast_run3; Containing 1001 trees. 



#### Step 3: compute embedding

In [19]:
# method can be chosen among pcoa, tsne, isomap, lle #
# pro tip: if the distance matrix has not been computed prior to calling this function, #
# it will automatically be computed using hashrf_RF #
beast_collection.embed(method="pcoa", dimensions=3, quality=False)

Output()

pcoa | Done!

#### Step 4: plot embeddings

In [20]:
beast_collection.plot_2D(
        method='pcoa', # method used for the embedding - if not previously computed, it will be computed when calling this function #
        save=False, # save the plot in a pdf format? The .html will be saved anyway! #
        name_plot=None, # specific name for the plot #
        static=False, # create a static element rather than a dynamic widget #
        plot_meta="SET-ID", # meta-variable used to colour the points (trees) #
        plot_set=None, # select specific set of sets of trees in the collection for the plot #
        select=False, # create widgets to hide/display tree_sets in the graph #
        same_scale=False,) # use the same colorscale for each tree_set (useful when the same metric is compared among sets) #

In [21]:
beast_collection.plot_3D(
        method='pcoa',
        save=False,
        name_plot=None,
        static=False,
        plot_meta="SET-ID",
        plot_set=None,
        select=False,
        same_scale=False,
        z_axis=None,) # option to substitute the 3rd axis (Z-axis/3rd PCoA) with a meta-variable of choice #

***
# Bootstrap analysis
### <font color='purple'> using ML and bootstrap trees from RAxML </font> 

The trees used in this example are derived from analysis of many genomic loci over a common set of 89 mammals. We do not delve into the biological interpretation of this specific example, as we discussed that in our manuscript. <br> Instead, we point your attention to some specific features of pear's interactive representations: <ul>
    <li> First: you can directly plot the `PCoA` embedding of the Robinson Foulds distances computed with `hashrf` without explicitly going through these steps. In fact, running a plotting function bypasses the need to specify distance metric calculation and embedding steps, performing them "under the hood" if needed;
    <li> In the 2D representation:<ul>
        <li> From the dropdown menu, you can choose the meta-vatiable used to color the points in the graphs;
        <li> The red button "Save plot as pdf" saves the plot in a `.pdf` file;
        <li> The $#show `name of the tree set`$ buttons allow showing and hiding specific sets of trees;
        <li> You can choose between the scatter representation or a contour plot;
        <li> While visualizing a contour plot, you can choose to hide or show the points (trees) with the light blue button toggling "Show points on a Contour plot";
        <li> You can choose between visualizing only the points or also the connections between them (indicating the progress through the sequential order of each tree set - not meaningful for bootstrap sets, but potentially helpful for e.g. MCMC runs or ML searches);
        <li> You can use the native tools of the plot to zoom in to focus on a specific area of the graph;
        <li> If you want to focus attention on specific sets of trees, click on them: this will highlight the selected set and make the other sets transparent. You can continue and highlight other sets by simply clicking on them. In order to go back to the original representation, just need click on one of the previously highlighted sets for a second time.
    </ul>
   <li> In the 3D representation:<ul>
       <li> Same features of the 2D representation. However, there is no contour plot in this case.

In [93]:
input_dir = '../bootstrap_mammals/'
# use bootstrap and ML trees derived from 6 different genomic loci #
files = ([f'bootstrap_{i}' for i in [6086, 229, 4544, 281, 694, 10409]] + 
      [f'bestTree_{i}' for i in [6086, 229, 4544, 281, 694, 10409]])    
   # these following loci were used in an earlier version of our ms. #
   # files = ([f'bootstrap_{i}' for i in [6086, 5261, 5092, 281, 11289, 10409]] +  #
   #     [f'bestTree_{i}' for i in [6086, 5261, 5092, 281, 11289, 10409]]) #
collection = tree_set.set_collection([os.path.join(input_dir, f) for f in files])
print(collection)

─────────────────────────────            
 Tree set collection containing 3006 trees;            
 File: Set_collection_ce9dd53a-4e11-459c-8059-531825821e66;
 Distance matrix: not computed.                
───────────────────────────── 
bootstrap_6086; Containing 500 trees. 
bootstrap_229; Containing 450 trees. 
bootstrap_4544; Containing 400 trees. 
bootstrap_281; Containing 600 trees. 
bootstrap_694; Containing 500 trees. 
bootstrap_10409; Containing 550 trees. 
bestTree_6086; Containing 1 trees. 
bestTree_229; Containing 1 trees. 
bestTree_4544; Containing 1 trees. 
bestTree_281; Containing 1 trees. 
bestTree_694; Containing 1 trees. 
bestTree_10409; Containing 1 trees. 



In [94]:
collection.plot_2D('pcoa', select=True)

Output()

hashrf_RF | Done!

Output()

pcoa | Done!

In [96]:
collection.plot_3D('pcoa', select = True)

****
# Maximum Likelihood search
### <font color='purple'> using MAPLE with different starting trees </font> 

In this example we look at ML tree search trajectories in <a href="https://www.nature.com/articles/s41588-023-01368-0">MAPLE</a>, and we focus the reader's attention on the possibility of specifying a metadata file to add meta-variable data and use them to color the points in the plots or to be displayed on the $Z$-axis in the 3D plot.<br>
In the cell below we specify a list of files containing trees and a list of files containing the likelihood of each tree.

In [97]:
maple_tree = [ # 4 MAPLE runs (trajectories) on the same SARS-CoV-2 alignment but with different starting trees #
    "IQtreeStartingTree_slowest_Trees",
    "ParsimonyRAxMLStartingTree_GTRmodel_Trees",
    "RAxMLNGStartingTree_Trees",
    "UshERStartingTree_Trees",
    ]

maple_LK = [ # likelihoods for the trees in each trajectory #
    "IQtreeStartingTree_slowest_LK",
    "ParsimonyRAxMLStartingTree_GTRmodel_LK",
    "RAxMLNGStartingTree_LK",
    "UshERStartingTree_LK",
]

maple_dir = '../MAPLE_res/'

In the cell below we check that "smart_RF_DistMat_Maple.csv", a precomputed distance matrix, is present in the folder. (Note the status reported after "Distance matrix:") We precomputed the distances using the `smart_RF` algorithm.  This is an implementation of the Robinson Foulds metric which understands that very short branches are sometimes used to indicate multifurcations in a phylogeny.  `smart_RF` takes much longer than `hashrf`. 

In [98]:
set_list = [os.path.join(maple_dir, tree) for tree in maple_tree]
try:collection_maple = tree_set.set_collection(set_list, distance_matrix = "../MAPLE_res/smart_RF_DistMat_Maple.csv")
except:collection_maple = tree_set.set_collection(set_list)
finally:print(collection_maple)

─────────────────────────────            
 Tree set collection containing 136 trees;            
 File: Set_collection_1b58a7ef-a2c7-46b8-ae8b-075422872190;
 Distance matrix: computed.                
───────────────────────────── 
IQtreeStartingTree_slowest_Trees; Containing 33 trees. 
ParsimonyRAxMLStartingTree_GTRmodel_Trees; Containing 47 trees. 
RAxMLNGStartingTree_Trees; Containing 26 trees. 
UshERStartingTree_Trees; Containing 30 trees. 



This is how we extracted the likelihood metadata information from multiple files - you can skip this step as we then saved the information in the provided "Likelihoods.csv" file. <br> 
**Note** that the likelihoods are linearly shifted and scaled to have range 0-1.

In [99]:
LKs = dict()
for lk_file in maple_LK:
    file = os.path.join(maple_dir, lk_file)
    with open(file, 'r') as f:
        LKs[lk_file] = np.array(f.readlines())
        f.close()

LK = list()
for lk in LKs.values(): LK.extend(lk)

LK = np.array([tree.replace('\n', '') for tree in LK], dtype=np.float64)
# LK = np.concatenate([LK,np.array([-257513])], axis = 0) # I think we're abandoning this value for the best tree #
LK = np.interp(LK, (LK.min(), LK.max()), (0, +1)) # scale LKs between 0 and 1 #

df_LK = pd.DataFrame({'LK':LK})
df_LK.to_csv("../MAPLE_res/Likelihoods.csv")
df_LK

,LK
0,0.982114
1,0.982671
2,0.983253
3,0.983973
4,0.984479
...,...
131,0.998798
132,0.998836
133,0.998841
134,0.998967


There are two ways to introduce new variables in a <font color="red"><b>set_collection</b></font>'s or <font color="blue"><b>tree_set</b></font>'s metadata:

In [100]:
# Method 1: #
# Given that metadata is a pandas dataframe, #
# one may simply add columns to it! #
df_LK = pd.read_csv("../MAPLE_res/Likelihoods.csv")
collection_maple.metadata['LK'] = df_LK['LK']
collection_maple.metadata

,SET-ID,STEP,LK
0,IQtreeStartingTree_slowest_Trees,0,0.982114
1,IQtreeStartingTree_slowest_Trees,1,0.982671
2,IQtreeStartingTree_slowest_Trees,2,0.983253
3,IQtreeStartingTree_slowest_Trees,3,0.983973
4,IQtreeStartingTree_slowest_Trees,4,0.984479
...,...,...,...
131,UshERStartingTree_Trees,25,0.998798
132,UshERStartingTree_Trees,26,0.998836
133,UshERStartingTree_Trees,27,0.998841
134,UshERStartingTree_Trees,28,0.998967


In [101]:
# Method 2: #
# One may specify a file with additional #
# metadata when the object is created #
collection_maple = tree_set.set_collection(set_list, distance_matrix = "../MAPLE_res/smart_RF_DistMat_Maple.csv", metadata="../MAPLE_res/Likelihoods.csv")
collection_maple.metadata

,SET-ID,STEP,LK
0,IQtreeStartingTree_slowest_Trees,0,0.982114
1,IQtreeStartingTree_slowest_Trees,1,0.982671
2,IQtreeStartingTree_slowest_Trees,2,0.983253
3,IQtreeStartingTree_slowest_Trees,3,0.983973
4,IQtreeStartingTree_slowest_Trees,4,0.984479
...,...,...,...
131,UshERStartingTree_Trees,25,0.998798
132,UshERStartingTree_Trees,26,0.998836
133,UshERStartingTree_Trees,27,0.998841
134,UshERStartingTree_Trees,28,0.998967


If, for any reason, the "smart_RF_DistMat_Maple.csv" file is not present in the folder, we need to compute the distance matrix. Above we chose to upload the precomputed distance matrix as the `smart_RF` method takes much longer to compute than `hashrf_RF`, and we wanted this notebook to run smoothly and rapidly, but here is now to compute the distances if needed:

In [102]:
if collection_maple.distance_matrix is None:
    start = time.time()
    collection_maple.calculate_distances('smart_RF')
    np.savetxt("../MAPLE_res/smart_RF_DistMat_Maple.csv", collection_maple.distance_matrix, delimiter=",")
    print(time.time() - start)
# confirm that we now have a distance matrix of the correct size: #
collection_maple.distance_matrix.shape

(136, 136)

First a simple 3D plot, which shows the different ML searches starting further apart and converging, although they do not reach a common ML tree (presumably because of the difficulty of analyzing large SARS-CoV-2 data sets).

In [103]:
collection_maple.plot_3D('pcoa', select = True)

pcoa | Done!

Next we colour the trees based on their likelihood in the 2D plot; in the 3D plot we replace the $Z$-axis with the likelihood. <br>
`same_scale` makes sure that points with the same value have the same colour in the graphs. <br> 
Notice that the trees from one run (starting tree from RAxML) have markedly lower likelihoods.

In [108]:
collection_maple.plot_2D('pcoa', plot_meta = 'LK', same_scale = True, select = True)

In [109]:
collection_maple.plot_3D('pcoa', plot_meta = 'LK', same_scale = True, select = True, z_axis = 'LK')

What if I wanted to make one of these trees to "pop out"? Well, we can add a coulumn called `"highlight"` that will automatically be read by the plotting function to "highlight" the points specified. The column has to be binary (0s and 1s), where the 1s indicate that a tree should be highlighted. It is crucial here to know the order and size of the sets, as the column is shared by the whole collection. <BR> 
    <b>Note</b> that the vector `"highlight"` <b>must be integer (dtype = int)!</b>.

#### Easy example: 
we want to highight the tree with highest likelihood (scaled value of 1), which is the 33rd in the collection.

In [118]:
pd.options.display.max_rows = 200 # allow notebook to show up to 200 lines of output before truncating #
highlight_mask = np.zeros(collection_maple.metadata.shape[0], dtype=int) # vector of 0s with size = n_trees #
highlight_mask[32] = 1 # 33rd element set to 1
collection_maple.metadata['highlight'] = highlight_mask
collection_maple.metadata

,SET-ID,STEP,LK,highlight
0,IQtreeStartingTree_slowest_Trees,0,0.982114,0
1,IQtreeStartingTree_slowest_Trees,1,0.982671,0
2,IQtreeStartingTree_slowest_Trees,2,0.983253,0
3,IQtreeStartingTree_slowest_Trees,3,0.983973,0
4,IQtreeStartingTree_slowest_Trees,4,0.984479,0
5,IQtreeStartingTree_slowest_Trees,5,0.984984,0
6,IQtreeStartingTree_slowest_Trees,6,0.985619,0
7,IQtreeStartingTree_slowest_Trees,7,0.986440,0
8,IQtreeStartingTree_slowest_Trees,8,0.986821,0
9,IQtreeStartingTree_slowest_Trees,9,0.988238,0


In [119]:
collection_maple.plot_2D('pcoa', plot_meta = 'LK', same_scale = True, select = False, static = True)

#### Hard (not really) example:
Suppose we want to highight the last tree of each set.
In order to perform this task, we can exploit an additional tool: the `set_collection.data`.
This object contains some useful information regarding the structure of each set composing the collection. In particular, we may be interested in the `n_trees` variable which can be used to assess the size of each set! 

In [121]:
# this is the object --> collection_maple.data # Note that this is a nested dictionary #
for Tset, info in collection_maple.data.items():
    print(Tset, f"has {info['n_trees']} trees")

IQtreeStartingTree_slowest_Trees has 33 trees
ParsimonyRAxMLStartingTree_GTRmodel_Trees has 47 trees
RAxMLNGStartingTree_Trees has 26 trees
UshERStartingTree_Trees has 30 trees


In [126]:
highlight_mask = np.zeros(collection_maple.metadata.shape[0], dtype=int)
last_tree = 0
for Tset, info in collection_maple.data.items():
    last_tree += info["n_trees"]
    highlight_mask[last_tree - 1] = 1
collection_maple.metadata['highlight'] = highlight_mask
collection_maple.metadata

,SET-ID,STEP,LK,highlight
0,IQtreeStartingTree_slowest_Trees,0,0.982114,0
1,IQtreeStartingTree_slowest_Trees,1,0.982671,0
2,IQtreeStartingTree_slowest_Trees,2,0.983253,0
3,IQtreeStartingTree_slowest_Trees,3,0.983973,0
4,IQtreeStartingTree_slowest_Trees,4,0.984479,0
5,IQtreeStartingTree_slowest_Trees,5,0.984984,0
6,IQtreeStartingTree_slowest_Trees,6,0.985619,0
7,IQtreeStartingTree_slowest_Trees,7,0.986440,0
8,IQtreeStartingTree_slowest_Trees,8,0.986821,0
9,IQtreeStartingTree_slowest_Trees,9,0.988238,0


In [127]:
collection_maple.plot_2D('pcoa', plot_meta = 'LK', same_scale = True, select = False)